### In the knowledge base, you should include: 
- Concepts (classes) - at least 15: Both atomic and composed (by EXISTS, FILLS, ALL, AND),
- Individuals (Instances) - at least 15, 
- Properties: at least 10: with suitable domain and range, some of them should be functional, transitive, or inverse of other properties in the KB.

### Give examples of reasoning over the knowledge base: 
- KB |= (c → e)  - at least 1,
- KB |= (c ⊑ d) - at least 1,
- Classification - at least 1.

In [338]:
# imports
from owlready2 import *

owlready2.JAVA_EXE = "C:\\Program Files\\java\\jdk-17\\bin\\java.exe"

# Create ontology

In [339]:
# create new ontology
game_ontology = get_ontology("http://test.org/game_ontology.owl")
game_ontology.destroy()
game_ontology = get_ontology("http://test.org/game_ontology.owl")

## Atomic concepts and their properties

In [340]:
with game_ontology:
    # Classes (1 - 5)
    class Game(Thing):
        pass
    class Field(Thing):
        pass
    class Equipment(Thing):
        pass    
    class Spectator(Thing):
        pass
    class Player(Thing):
        pass

    # Properties (1 - 12)
    class played_on(Game >> Field):
        pass

    class require(Game >> Equipment):
        pass

    class has_players_number(Game >> int):
        pass

    class has_duration(Game >> int):
        pass

    class is_professional(Game >> bool, FunctionalProperty):
        pass
    class is_amateur(Game >> bool, FunctionalProperty):
        Not(is_professional)
        pass

    class plays(Player >> Game):
        pass
    class is_played_by(Game >> Player):
        inverse_property = plays
        pass

    class watch(Spectator >> Game):
        pass
    class watched_by(Game >> Spectator):
        inverse_property = watch
        pass
    class has_interest_in(Spectator >> Game):
        pass
    class has_to_remain_silent(Spectator >> bool, FunctionalProperty):
        pass

## Composed concepts and their properties

### Fields

In [341]:
with game_ontology:
    # Classes (6 - 7)
    class Lawn(Field):
        pass
    class Board(Field):
        pass    

    # Properties (13 - 13)
    class is_custom_board(Board >> bool, FunctionalProperty):
        pass

### Equipments

In [342]:
with game_ontology:
    # Classes (8 - 16)
    class Table(Equipment):
        pass
    class Card(Equipment):
        pass
    class Dice(Equipment):
        pass
    class Pawn(Equipment):
        pass
    class Paper(Equipment):
        pass
    class Pencil(Equipment):
        pass
    class Computer(Equipment):
        pass
    class SportEquipment(Equipment):
        pass
    class Ball(SportEquipment):
        pass

    # Properties (14 - 21)
    class is_normal_card(Card >> bool, FunctionalProperty):
        pass
    class is_custom_card(Card >> bool, FunctionalProperty):
        Not(is_normal_card)
        pass

    class has_sides(Dice >> int, FunctionalProperty):
        pass
    class has_number(Dice >> int, FunctionalProperty):
        pass
    class has_greater_number_than(Dice >> Dice, TransitiveProperty):
        pass
    class has_lower_number_than(Dice >> Dice, TransitiveProperty):
        inverse_property = has_greater_number_than
        pass
    
    class is_blank(Paper >> bool, FunctionalProperty):
        pass
    class has_color(Pencil >> str):
        pass

### Spectators

In [343]:
with game_ontology:
    # Classes (17 - 18)
    class Fan(Spectator):
        equivalent_to = [Spectator
                        & has_interest_in.only(Game)]
        pass
    class Kibitzer(Spectator):
        equivalent_to = [Spectator
                        & watch.only(Game)
                        & has_to_remain_silent.value(True)]

    # Properties (22 - 23)
    class has_favourite(Fan >> Player):
        pass
    class has_fans(Player >> Fan):
        inverse_property = has_favourite
        pass

### Games by number of players

In [344]:
with game_ontology:
    # Classes (19 -20)
    class SinglePlayerGame(Game):
        equivalent_to = [Game
                        & is_played_by.only(Player)
                        & has_players_number.exactly(1)] 
        pass
    class MultiPlayerGame(Game):
        equivalent_to = [Game
                        & is_played_by.some(Player)
                        & has_players_number.min(2)]        
        pass
    
    # Properties (24 - 25)
    class has_team(MultiPlayerGame >> bool, FunctionalProperty):
        pass
    class has_opponent(MultiPlayerGame >> bool, FunctionalProperty):
        pass

### Games by activity

In [345]:
with game_ontology:
    # Classes (21 - 25)
    class IntelectualGame(Game):
        pass
    class LogicalGame(IntelectualGame):
        pass
    class StrategyGame(IntelectualGame):
        pass
    class KnowledgeGame(IntelectualGame):
        pass
    class PhysicalActivityGame(Game):
        pass
        
    # Properties (-)
    ...

### Sport Games

In [346]:
with game_ontology:
    # Classes (26 - 31)
    class TeamGame(Game):
        equivalent_to = [MultiPlayerGame
                        & has_team.value(True)]      
        pass
    class SportGame(PhysicalActivityGame):
        equivalent_to = [PhysicalActivityGame
                        & require.some(SportEquipment)
                        & watched_by.some(Spectator)]
        pass
    class IndividualSportGame(SinglePlayerGame, SportGame):
        pass
    class TeamSportGame(TeamGame, SportGame):
        pass
    class ProfessionalSportGame(SportGame):
        equivalent_to = [SportGame
                        & is_professional.value(True)] 
        pass 
    class ProfessionalBallGameOnLawn(ProfessionalSportGame):
        equivalent_to = [ProfessionalSportGame
                        & require.only(Ball)
                        & played_on.only(Lawn)]
        pass
    
    # Properties (26 - 26)
    class has_referee(ProfessionalSportGame >> bool, FunctionalProperty):
        pass

### Cards

In [347]:
with game_ontology:
    # Classes (32 - 35)
    class PlayingCard(Card):
        equivalent_to = [Card
                        & is_normal_card.value(True)] 
        pass
    class MagicCard(Card):
        equivalent_to = [Card
                        & is_custom_card.value(True)]
        pass

    class CardSuit(Thing):
        pass
    class CardValue(Thing):
        pass

    # Properties (27 - 32)    
    class has_suit(PlayingCard >> CardSuit, FunctionalProperty):
        pass
    class has_value(PlayingCard >> CardValue, FunctionalProperty):
        pass
    class has_greater_value_than(PlayingCard >> PlayingCard, TransitiveProperty):
        pass
    class has_lower_value_than(PlayingCard >> PlayingCard, TransitiveProperty):
        inverse_property = has_greater_value_than
        pass

    class has_type(MagicCard >> str, FunctionalProperty):
        pass
    class has_effect(MagicCard >> str):
        pass

suits = [CardSuit("Spades"), CardSuit("Hearts"), CardSuit("Diamonds"), CardSuit("Clubs")]
CardSuit.is_a.append(OneOf(suits))

values = [CardValue("Two"), CardValue("Three"), CardValue("Four"), CardValue("Five"),
          CardValue("Six"), CardValue("Seven"), CardValue("Eight"), CardValue("Nine"),
          CardValue("Ten"), CardValue("Jack"), CardValue("Queen"), CardValue("King"), CardValue("Ace")]
CardValue.is_a.append(OneOf(values))

### Table Games

In [348]:
with game_ontology:
    # Classes (36 - 41)
    class VideoGame(Game):
        equivalent_to = [Game
                        & require.only(Table & Computer)]
        pass
    class TableTopGame(LogicalGame, StrategyGame):
        equivalent_to = [LogicalGame
                        & StrategyGame
                        & (MultiPlayerGame | SinglePlayerGame)
                        & require.some(Table)]
        pass
    class BoardGame(TableTopGame):
        equivalent_to = [TableTopGame
                        & played_on.only(Board)]
        pass
    class DiceGame(TableTopGame):
        equivalent_to = [TableTopGame
                        & require.only(Dice)]
        pass
    class PaperAndPencilGame(TableTopGame):
        equivalent_to = [TableTopGame
                        & require.only(Paper)
                        & require.only(Pencil)]
        pass
    class CardGame(TableTopGame):
        equivalent_to = [TableTopGame
                        & require.only(Card)]
        pass
    
    # Properties (33 - 33)
    class need(require):
        domain    = [BoardGame]
        range     = [MagicCard | Dice | Pawn | Paper | Pencil]
        pass

# Instances

In [349]:
# Instances - 31
table = Table("table")
chess_board = Board("board", is_custom_board = True)

green = Lawn("green")
golf_ball = Ball("golf_ball")
golf_stick = Equipment("golf_stick")
golf = ProfessionalBallGameOnLawn(played_on = [green], require = [golf_ball, golf_stick])

football = ProfessionalBallGameOnLawn("football", has_team = True, has_referee = True, has_duration = [90])

go = BoardGame("go", has_players_number = [2], has_opponent = True)

pawn = Pawn("pawn")
knight = Pawn("knight")
bishop = Pawn("bishop")
rook = Pawn("rook")
queen = Pawn("queen")
king = Pawn("king")
chess = BoardGame("chess", has_players_number = [2], need = [pawn, knight, bishop, rook, queen, king], played_on = [chess_board], require = [table])

pokemon_go = VideoGame("Pokemon Go")

north = Player("Yavor")
east = Player("Sasho")
south = Player("Tanya")
west = Player("Gosho")

kibitzer = Kibitzer("Pesho", plays = [pokemon_go])
fan = Fan("Tsetso", has_favourite = [east, west], plays = [chess, pokemon_go])

bridge = CardGame("bridge", has_players_number = [4], is_played_by = [north, east, south, west], watched_by = [fan, kibitzer], require = [PlayingCard])

king_of_spades = PlayingCard("King of Spades", has_suit = CardSuit.instances()[0], has_value = CardValue.instances()[11])
queen_of_hearts = PlayingCard("Queen of Hearts", has_suit = CardSuit.instances()[1], has_value = CardValue.instances()[10], has_lower_value_than = [king_of_spades])
jack_of_clubs = PlayingCard("Jack of Clubs", has_suit = CardSuit.instances()[3], has_value = CardValue.instances()[9], has_lower_value_than = [king_of_spades, queen_of_hearts])
ace_of_spades = PlayingCard("Ace of Spades", has_suit = CardSuit.instances()[0], has_value = CardValue.instances()[12], has_greater_value_than = [king_of_spades])

tic_tac_toe = PaperAndPencilGame("tic_tac_toe", has_players_number = [2], is_amateur = True, has_opponent = True)

dice_6 = Dice("6 sided dice", has_sides = 6, has_number = 5) 
dice_12 = Dice("12 sided dice", has_sides = 12, has_number = 3, has_lower_number_than = [dice_6])
dice_8 = Dice("8 sided dice", has_sides = 8, has_number = 7, has_greater_number_than = [dice_6, dice_12])

backgammon = BoardGame("backgammon", require = [dice_6], has_opponent = True).is_a.append(MultiPlayerGame)

catan = BoardGame("Catan", require = [MagicCard], has_team = False)

test_card = Card("test_card", is_normal_card = True)

# Test

In [350]:
print(king_of_spades.has_greater_value_than)
print(ace_of_spades.INDIRECT_has_greater_value_than)
print(jack_of_clubs.INDIRECT_has_lower_value_than)

print(dice_12.has_greater_number_than)
print(dice_12.has_lower_number_than)

print(CardValue.instances())

[game_ontology.Queen of Hearts, game_ontology.Jack of Clubs]
[game_ontology.Queen of Hearts, game_ontology.Jack of Clubs, game_ontology.King of Spades, game_ontology.Card & game_ontology.is_normal_card.value(True)]
[game_ontology.Queen of Hearts, game_ontology.Ace of Spades, game_ontology.King of Spades, game_ontology.Card & game_ontology.is_normal_card.value(True)]
[]
[game_ontology.6 sided dice, game_ontology.8 sided dice]
[game_ontology.Two, game_ontology.Three, game_ontology.Four, game_ontology.Five, game_ontology.Six, game_ontology.Seven, game_ontology.Eight, game_ontology.Nine, game_ontology.Ten, game_ontology.Jack, game_ontology.Queen, game_ontology.King, game_ontology.Ace]


### Reasoning

In [351]:
with game_ontology:
    
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)
    # sync_reasoner(infer_property_values = True)
    # list(game_ontology.inconsistent_classes())
    print(game_ontology.get_parents_of(test_card), test_card.__class__)
    print(game_ontology.get_instances_of(Game))
    print(game_ontology.get_children_of(Card))

* Owlready2 * Running Pellet...
    C:\Program Files\java\jdk-17\bin\java.exe -Xmx2000M -cp C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Users\Tanya\AppData\Loc

* Owlready * Adding relation game_ontology.chess require game_ontology.king
* Owlready * Adding relation game_ontology.chess require game_ontology.pawn
* Owlready * Adding relation game_ontology.chess require game_ontology.knight
* Owlready * Adding relation game_ontology.chess require game_ontology.bishop
* Owlready * Adding relation game_ontology.chess require game_ontology.rook
* Owlready * Adding relation game_ontology.chess require game_ontology.queen
* Owlready * Adding relation game_ontology.football is_professional true
* Owlready * Adding relation game_ontology.professionalballgameonlawn25 is_professional true
* Owlready * Adding relation game_ontology.Pesho has_to_remain_silent true
[game_ontology.Card, game_ontology.PlayingCard] game_ontology.PlayingCard
[]
[game_ontology.PlayingCard, game_ontology.MagicCard]


* Owlready2 * Pellet took 1.6633186340332031 seconds
* Owlready * Equivalenting: game_ontology.Fan game_ontology.Spectator
* Owlready * Equivalenting: game_ontology.Spectator game_ontology.Fan
* Owlready * Reparenting game_ontology.test_card: {game_ontology.Card} => {game_ontology.PlayingCard}
* Owlready * Reparenting game_ontology.golf_stick: {game_ontology.Equipment} => {game_ontology.Ball}
* Owlready * Reparenting game_ontology.go: {game_ontology.BoardGame} => {game_ontology.MultiPlayerGame, game_ontology.BoardGame}
* Owlready * Reparenting game_ontology.Catan: {game_ontology.BoardGame} => {game_ontology.MultiPlayerGame, game_ontology.BoardGame}
* Owlready * Reparenting game_ontology.tic_tac_toe: {game_ontology.PaperAndPencilGame} => {game_ontology.PaperAndPencilGame, game_ontology.MultiPlayerGame}
* Owlready * Reparenting game_ontology.TeamGame: {game_ontology.Game} => {game_ontology.MultiPlayerGame}
* Owlready * Reparenting game_ontology.football: {game_ontology.ProfessionalBallGa

# Save

In [352]:
game_ontology.save("../game_ontology.owl")